In [ ]:
import ipywidgets as widgets
from ipywidgets import GridspecLayout, Layout, HBox, VBox
from datetime import datetime, timedelta
from IPython.display import IFrame, display
from attestator import generate_pdf
import pickle
import io
#import PyPDF2

In [ ]:
# Define all widgets
use_defaults = False
try:
    with open('user_data', 'rb') as f:
        user_data = pickle.load(f)
    date_str = str(user_data["birth_year"]) +'-'+ str(user_data["birth_month"]) +'-'+ str(user_data["birth_day"])
    saved_birth_date = datetime.strptime(date_str, '%Y-%m-%d')
except:
    use_defaults = True

largeur = '70%'

first_name = widgets.Text(
    value='' if use_defaults else user_data["first_name"],
    placeholder='Jean',
    description='Prénom:',
    disabled=False,
    layout = Layout(width=largeur)
)

last_name = widgets.Text(
    value='' if use_defaults else user_data["last_name"],
    placeholder='Dupond',
    description='Nom:',
    disabled=False,
    layout = Layout(width=largeur)
)

birth_date = widgets.DatePicker(
    value = datetime.now() if use_defaults else saved_birth_date,
    description='Né le:',
    disabled=False,
    layout = Layout(width=largeur)
)

birth_place = widgets.Text(
    value='' if use_defaults else user_data["birth_place"],
    placeholder='Paris',
    description='À:',
    disabled=False,
    layout = Layout(width=largeur)
)

address = widgets.Text(
    value='' if use_defaults else user_data["address"],
    placeholder='666 avenue de France',
    description='Adresse:',
    disabled=False,
    layout = Layout(width=largeur)
)

zip_code = widgets.Text(
    value='' if use_defaults else user_data["zip_code"],
    placeholder='75000',
    description='Code postal:',
    disabled=False,
    layout = Layout(width=largeur)
)

city = widgets.Text(
    value='' if use_defaults else user_data["city"],
    placeholder='Paris',
    description='Ville:',
    disabled=False,
    layout = Layout(width=largeur)
)

reasons = widgets.SelectMultiple(
    options=['travail', 'courses', 'sante', 'famille', 'sport', 'judiciaire', 'missions'],
    value=['courses'] if use_defaults else user_data["reasons"],
    description='Motif de sortie:',
    disabled=False,
    layout = Layout(width=largeur),
    rows = 7,
)

attestation = widgets.Button(
    description='Créer attestation',
    disabled=False,
    button_style='danger',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Créer l'attestation au format pdf",
    icon='check',
    layout=Layout(width='60%', margin='2% 0 0 0'),
)


out = widgets.Output()

w_list = [first_name, last_name, birth_date, birth_place,
          address, zip_code, city, reasons, attestation]

In [ ]:
# Arrange all widgets in a 5x2 grid
grid = GridspecLayout(6, 2, width='70%')

for i in range(2):
    for j in range(2):
        grid[i, j] = w_list[i*2+j]
for i in range(3,6):
    grid[i, 0] = w_list[i+1]

grid[3:,1] = reasons

grid = VBox([grid, attestation])

grid = VBox([grid, out])

In [ ]:
# met à jour les données    
now = datetime.now()
user_data = {}

def update_user_data():
    global user_data
    user_data = {
    "first_name": first_name.value,
    "last_name": last_name.value,
    "birth_year": birth_date.value.year,
    "birth_month": birth_date.value.month,
    "birth_day": birth_date.value.day,
    "birth_place": birth_place.value,
    "address": address.value,
    "zip_code": zip_code.value,
    "city": city.value,
    "reasons": reasons.value,
    "trip_datetime": datetime.now() + timedelta(minutes=10),
    }

In [ ]:
def on_attestation_clicked(b):
    with out:
        out.clear_output()
        try:
            update_user_data()     
            #attest = io.BytesIO()
            generate_pdf(user_data, output_file="attestation.pdf")
            #display(IFrame("attestation.pdf", width=800, height=300))
            
            with open('user_data', 'wb') as f: #save parameters to disk
                pickle.dump(user_data, f)
        except AssertionError:
            print("Erreur: formulaire incomplet.")

attestation.on_click(on_attestation_clicked)

In [ ]:
grid